# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [233]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [234]:
data = "City_Data.csv"

weather_data = pd.read_csv(data)

weather_data.head()


,City,Lat,Long,Temp,Max Temp,Min Temp,Humidity,Clouds,Wind,Country,Datetime
0,dikson,73.5069,80.5464,5.92,5.92,5.92,97,100,5.08,RU,1660230932
1,vaini,-21.2000,-175.2000,21.09,21.09,21.09,100,75,4.12,TO,1660230932
2,punta arenas,-53.1500,-70.9167,2.98,2.98,2.98,72,20,2.26,CL,1660230689
3,riga,57.0000,24.0833,26.71,27.24,26.04,35,0,2.06,LV,1660230913
4,puerto ayora,-0.7393,-90.3518,21.15,21.15,21.15,85,100,4.47,EC,1660230933


In [235]:
locations = weather_data[["Lat", "Long"]]
print(locations)

         Lat      Long
0    73.5069   80.5464
1   -21.2000 -175.2000
2   -53.1500  -70.9167
3    57.0000   24.0833
4    -0.7393  -90.3518
..       ...       ...
554   8.2000  -59.7833
555  63.9333   31.6500
556  28.2000   78.3833
557  63.9242  127.4739
558 -19.8000  -63.2167

[559 rows x 2 columns]


In [236]:
rating = weather_data['Humidity'].astype(float)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [237]:
fig = gmaps.figure(center = (0,30), zoom_level=2, map_type='HYBRID')

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [256]:
# Holiday destination filtering variables

max_temp = 35
min_temp = 25
max_wind_speed = 4
max_cloudiness = 0

holiday_data  = weather_data.loc[(weather_data['Max Temp'] <= max_temp) &
                                (weather_data['Min Temp'] >= min_temp) &
                                (weather_data['Wind'] <= max_wind_speed) &
                                (weather_data['Clouds'] <= max_cloudiness), :]

holiday_data = holiday_data.reset_index(drop=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [257]:
# Creating a new dataframing and assigning it the columns City and Country from the holiday_data dataframe
hotel_df = pd.DataFrame().assign(City=holiday_data['City'], Country=holiday_data['Country'])

In [258]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   City     13 non-null     object
 1   Country  13 non-null     object
dtypes: object(2)
memory usage: 336.0+ bytes


In [259]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in holiday_data.iterrows():

    # get restaurant type from df
    Lat = row['Lat']
    Long = row['Long']

    # add keyword to params dict
    params['location'] = f'{Lat}, {Long}'

    # assemble url and make API request
    print(f"Retrieving a Hotel in {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest Hotel in {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Long'] = results[0]['geometry']['location']['lng']

    except (KeyError, IndexError):
        print(f"Sorry we couldnt find any Hotels found in {row['City']}")
  
    print("------------")


Retrieving a Hotel in riga.
Closest Hotel in riga is Hotel Skanste.
------------
Retrieving a Hotel in kuryk.
Closest Hotel in kuryk is Gostinitsa Kuryk - Otel' Dlya Vsekh Kategoriy Kliyentov ..
------------
Retrieving a Hotel in vila velha.
Closest Hotel in vila velha is Hotel Vitória Palace.
------------
Retrieving a Hotel in wageningen.
Closest Hotel in wageningen is Fletcher Hotel-Restaurant De Wageningsche Berg.
------------
Retrieving a Hotel in beyneu.
Closest Hotel in beyneu is NUR.
------------
Retrieving a Hotel in ferreira do alentejo.
Closest Hotel in ferreira do alentejo is Rural Moments.
------------
Retrieving a Hotel in dentsville.
Closest Hotel in dentsville is Columbia Northeast Hotel.
------------
Retrieving a Hotel in iberia.
Closest Hotel in iberia is back to back rv park.
------------
Retrieving a Hotel in chiredzi.
Closest Hotel in chiredzi is Clevers Hotel Chiredzi.
------------
Retrieving a Hotel in qobustan.
Sorry we couldnt find any Hotels found in qobustan
-

In [242]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        28 non-null     object 
 1   Country     28 non-null     object 
 2   Hotel Name  27 non-null     object 
 3   Lat         27 non-null     float64
 4   Long        27 non-null     float64
dtypes: float64(2), object(3)
memory usage: 1.2+ KB


In [243]:
hotel_df = hotel_df.dropna()
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 27
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        27 non-null     object 
 1   Country     27 non-null     object 
 2   Hotel Name  27 non-null     object 
 3   Lat         27 non-null     float64
 4   Long        27 non-null     float64
dtypes: float64(2), object(3)
memory usage: 1.3+ KB


In [244]:
hotel_df.head()

,City,Country,Hotel Name,Lat,Long
0,riga,LV,Hotel Skanste,56.966889,24.120312
1,kuryk,KZ,Gostinitsa Kuryk - Otel' Dlya Vsekh Kategoriy ...,43.179050,51.672569
2,hamilton,US,Six Acres Bed & Breakfast,39.185938,-84.544310
3,menomonie,US,Best Western Plus Menomonie Inn & Suites,44.902375,-91.929498
4,abu ghosh,IL,Jerusalem Hills Guest House,31.808263,35.104939


In [245]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Long"]]

In [246]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

# Display figure
fig2 = gmaps.figure(center = (0,30), zoom_level=3, map_type='SATELLITE')
fig2.add_layer(marker_layer)
fig2.add_layer(heat_layer)
fig2

# Display figure


Figure(layout=FigureLayout(height='420px'))